# Identify relationships between Prostate cancer associated SNPs and genes

In [1]:
# Load the required packages
from neo4j import GraphDatabase
import pandas as pd

## Setting parameters and creating connection to Neo4J

In [20]:
# Set the genomic parameters
delta_bp = 10000
FC_cutoff = 1.5
FDR_cutoff = 0.0001

In [3]:
# Setup a connection to the database
driver = GraphDatabase.driver("bolt://localhost:7687")
session =  driver.session()

## Loading and modifying the reference set

In [3]:
# Read in the data of Farashi et al. that is used as a reference set
farashi = pd.read_excel("Input sets/Farashi/41568_2018_87_MOESM1_ESM-3.xls", header = 1)

# Remove the empty rows at the bottom
farashi = farashi[farashi["SNP ID"].notnull() & farashi["Target/assigned/e-Gene"].notnull()]

In [4]:
# Replace fault-inducing postfixes for snp data
farashi["SNP ID"] = farashi["SNP ID"].str.strip()
farashi["SNP ID"].replace("(_A)|(_C)", "", regex = True, inplace = True)

# Remove entries without valid rs identifiers
farashi.drop(farashi[farashi["SNP ID"].str.startswith("chr")].index, inplace = True)

### Updating gene names and SNP ID's in reference set

Not all gene names appear to be correct, and could therefore be found in PathwayStudio. These were therefore searched on https://www.ncbi.nlm.nih.gov/gene/ to identify their updaed names

In [5]:
# Create separate entries for cells with delimited genes
farashi = farashi.assign(gene=farashi["Target/assigned/e-Gene"].str.split(';')).explode('gene')

# Some gene names are outdated/erroneous. These are manually mapped to correct genes as based on ncbi.org/gene
gene_replacement_dictionary = { "MSMB1" : "MSMB",
                                "MSMB2" : "MSMB",
                                "NCOA4-1" : "NCOA4P1",
                                "NCOA4-3" : "NCOA4P3", 
                                "ANKRD5" : "ANKEF1", 
                                "C6orf228" : "SMIM13",
                                "c-MYC, FoxA1 binding" : "MYC",
                                "HoxB13" : "HOXB13",
                                "LASS2" : "CERS2",
                                "C10orf32" : "BORCS7",
                                "LOC100505761" : "RPARP-AS1",
                                "LOC100505495" : "PCAT19",
                                "WDR52" : "CFAP44",
                                "HCG4P6" : "HCG4B",
                                "LOC285830" : "HLA-F-AS1",
                                "RAB7L1" : "RAB29",
                                "LOC284578" : "MFSD4A-AS1",
                                "AGAP7" : "AGAP7P",
                                "C2orf43" : "LDAH"}

farashi["Target/assigned/e-Gene"] = farashi["Target/assigned/e-Gene"].replace(gene_replacement_dictionary)

# Strip whitespaces
farashi["SNP ID"] = farashi["SNP ID"].str.strip()

Similar to the genes, some of the SNP ID's need to be updated as well, as they have been merged with other rs identifiers

These invalid rs identifiers can be found in the next step, where dbSNP is queried.
These missing rs identifiers were subsequently queried on https://www.ncbi.nlm.nih.gov/snp/, where their updated rs identifier was retrieved.

Nonetheless, some rs identifiers could not be found on the dbsnp at all, even to map it. These were removed from the reference set.

In [6]:
# Create a mapping dictionary
snp_replacement_dictionary = {"rs565245309" : "rs10700825",
                              "rs397764955" : "rs11371876",
                              "rs567544918" : "rs143009074",
                              "rs68007409" : "rs58061354",
                              "rs576874987" : "rs2735090",
                              "rs56969947" : "rs5794883",
                              "rs71390080" : "rs35883900",
                              "rs397885676" : "rs35853071",
                              "rs563936332" : "rs11425106",
                              "rs570238728" :  "rs2735091",
                              "rs386572937" : "rs2735095",
                              "rs368454874" : "rs5875242",
                              "rs576956856" : "rs557303655",
                              "rs527768094" : "rs3115587",
                              "rs34421549" : "rs11281315",
                              "rs543833159" : "rs9261476",
                              "rs573341295" : "rs3083610",
                              "rs397841490" : "rs3839562",
                              "rs72562630" : "rs10643878",
                              "rs67276543" : "rs34884832",
                              "rs113645266" : "rs6557271",
                              "rs540840764" : "rs9278594",
                              "rs145076668" : "rs34837204",
                              "rs79588872" : "rs35538902",
                              "rs397847839" : "rs35826034",
                              "rs551993434" : "rs11371951",
                              "rs113130272" : "rs11153141",
                              "rs114376585" : "rs3096702",
                              "rs527588882" : "rs9278592",
                              "rs144721865" : "rs9368661",
                              "rs572291073" : "rs2571388",
                              "rs376201080" : "rs142474496",
                              "rs34948850" : "rs10688614",
                              "rs397887654" : "rs36076724",
                              "rs114473420" : "rs3135340",
                              "rs371043306" : "rs145380596",
                              "rs572943237" : "rs11421756",
                              "rs139078838" : "rs9501073",
                              "rs539183916" : "rs2437062",
                              "rs386410791" : "rs141020575",
                              "rs141507970" : "rs9267919",
                              "rs397823414" : "rs35850123",
                              "rs63475060" : "rs5875246",
                              "rs139104997" : "rs9261481",
                              "rs150282463" : "rs13137700",
                              "rs143466021" : "rs9269108",
                              "rs5875234" : "rs3058350"
                             }

not_found_dbsnp = {"rs77010356", "rs60284051", "rs563604877"}

# Strip whitespaces
farashi["SNP ID"] = farashi["SNP ID"].replace(snp_replacement_dictionary)

In [7]:
# Get the SNP properties from dbsnp
import myvariant
mv = myvariant.MyVariantInfo()
   
dbsnp = mv.querymany(list(set(farashi["SNP ID"])), scopes='dbsnp.rsid', fields='dbsnp', returnall = True)
if not_found_dbsnp == set(dbsnp["missing"]): print("Only known SNPs missing")

querying 1-1000...done.
querying 1001-1139...done.
Finished.
482 input query terms found dup hits:
	[('rs6785962', 3), ('rs57471960', 2), ('rs6096208', 2), ('rs2191139', 3), ('rs5832650', 5), ('rs1115
3 input query terms found no hit:
	['rs77010356', 'rs60284051', 'rs563604877']


In [8]:
# Merge the dbSNP data with the reference set
dbsnp_tab = pd.DataFrame(dbsnp["out"])

# Drop the rs ids that were not found
dbsnp_tab.drop(dbsnp_tab[dbsnp_tab["notfound"] == True].index, inplace = True)

dbsnp_tab["chromosome"] = dbsnp_tab["dbsnp"].apply(lambda x: x["chrom"])
dbsnp_tab["location"] = dbsnp_tab["dbsnp"].apply(lambda x: x["hg19"]["start"] if "hg19" in x.keys() else None)
dbsnp_tab["ref"] = dbsnp_tab["dbsnp"].apply(lambda x: x["ref"])
dbsnp_tab["alt"] = dbsnp_tab["dbsnp"].apply(lambda x: x["alt"])

# Drop entries that do not have a chromosome location
dbsnp_tab.drop(dbsnp_tab[dbsnp_tab["location"].isnull()].index, inplace = True)
dbsnp_tab["location"] = dbsnp_tab["location"].astype(int)

positives = farashi.merge(dbsnp_tab[["query", "chromosome", "location", "ref", "alt"]], how = "inner", left_on = "SNP ID", right_on = "query")

### Properties of the reference set

In [7]:
# TODO: Aantal genen en SNPs per studie laten zien
# Ook aantal genen per SNP laten zien

### Filtering the reference set

In [8]:
# Remove entries that are based on a single eQTL study, or that are found in the exon/coding region
eQTL_studies = ["Dadaev T. et al. 2018", "Grisanzio, C. et al.  2012", "X. xu et al. 2014", "Thibodeau S.N.  et al. 2015"]
coding_snps = ["Coding region", "exonic"]

### Properties of the modified reference set

In [9]:
nodes = session.run("Match (n) WHERE n.name IN " + str(list(set(farashi["Target/assigned/e-Gene"]))) + " RETURN n").value()

In [11]:
# Validate the mappings

## Gene expression data

Now that we have our reference set, we obtain the gene-expression data.
This data is obtained from a separate Next-Generation sequencing dataset, which compared the expression of men with prostate cancer, to that of men without prostate cancer.

In [9]:
# Extract the gene expression data
cols = pd.read_csv("Input sets/Gene expression data/design_matrix_prostate_unpaired.txt", delimiter = "\t")
reads = pd.read_csv("Input sets/Gene expression data/expression_matrix_prostate_clean.txt", delimiter = "\t")
# Read in the differential expression calculations
dex = pd.read_csv("Input sets/Gene expression data/Galaxy37-[edgeR_DGE_on_2__design_matrix_prostate_unpaired.txt_-_differentially_expressed_genes].tabular.annotated.txt", 
                  delimiter = "\t", index_col = 0, names = ["ENSEMBL", "gene", "logFC", "logCPM", "LR", "pValue", "FDR"], header = 0)
dex["ENSEMBL"].replace("(\.\d+)", "", regex = True, inplace = True)

reads["total"] = reads.sum(axis = 1, numeric_only = True)
reads["freq"] = ((reads[list(cols["samplename"])] > 0) * 1).sum(axis = 1)
reads["gene_ids"].replace("(\.\d+)", "", regex = True, inplace = True)

reads = reads.merge(dex, left_on = "gene_ids", right_on = "ENSEMBL", how = "outer")

In [10]:
# Initiate the connection to the database (first time you may have to run ensembl.download() and ensembl.index() first)
# To align with our reference set, we take a Ensembl version from 2019
from pyensembl import EnsemblRelease
ensembl = EnsemblRelease(92)

# Function to extract the data (and prevent superfluous queries)
def getEnsemblData(id):
    try:
        data = ensembl.gene_by_id(id)
        return pd.Series({"gene_name" : data.gene_name, 
                          "chromosome" : data.contig, 
                          "start" : data.start,
                          "stop" : data.end,
                          "protein_coding" : data.is_protein_coding})
    except ValueError:
        return pd.Series({"gene_name" : None, 
                          "chromosome" : None, 
                          "start" : None,
                          "stop" : None,
                          "protein_coding" : None})

# Remove the postifxes of the gene identifiers
reads["gene_ids"].replace("\.\d", "", regex = True, inplace = True)
reads[["gene_name", "chromosome", "start", "stop", "protein_coding"]] = reads["gene_ids"].apply(lambda x: getEnsemblData(x))

# Drop the mitochondrial genes and the entries that could not be found
reads.drop(reads[reads["chromosome"].isin(["MT", None])].index, inplace = True)

In [21]:
# Create a dataframe of all combinations of SNPs and genes that are on the same chromosome
positives["location"] = positives["location"].astype(int)
snp_candidates = positives[["SNP ID", "chromosome", "location"]].merge(reads[["gene_ids", "gene_name", "chromosome", "start", "stop"]], on = "chromosome", how = "inner")

# Select the pairs where the SNP window and the gene windows overlap
snp_candidates["candidate"] = snp_candidates.apply(lambda x: True if ((x["stop"] >= x["location"] - delta_bp and x["stop"] <= x["location"] + delta_bp) or 
                                                                      (x["start"] >= x["location"] - delta_bp and x["start"] <= x["location"] + delta_bp)) else False, axis = 1)
snp_candidates.drop(snp_candidates[snp_candidates["candidate"] == False].index, inplace = True)

In [16]:
# Split up the dataset into cancer and non-cancer samples
cancer = reads[["gene_ids"] + list(cols.samplename[cols.condition == "cancer"])].copy()
normal = reads[["gene_ids"] + list(cols.samplename[cols.condition == "normal"])].copy()

In [17]:
# Check whether all genes in the reference set are in the expression data
len(set(farashi["Target/assigned/e-Gene"]) - set(reads["gene_name"]))

In [18]:
# Calculate the fold change and p-values

## Query all the incoming and outgoing pathway data for the genes

In [62]:
# Get the incoming and outgoing paths of the genes
direct = pd.DataFrame(session.run("Match p = (n)-[r]-(x:Protein)" +
                                  " WHERE n.name IN " + str(list(farashi["Target/assigned/e-Gene"])) + 
                                  " RETURN startnode(r).name AS start," +
                                  " type(r) AS relationship," + 
                                  " r.pmid AS pmid," +
                                  " length(r.pmid) AS npmid," + 
                                  " endnode(r).name AS end").data())

In [63]:
# Get the indirect outgoing paths
outgoing = pd.DataFrame(session.run("Match p = (n)-[r]->(x:Protein)-[r2]->(y:Protein)" +
                                    " WHERE n.name IN " + str(list(set(farashi["Target/assigned/e-Gene"]))) +
                                    " RETURN startnode(r).name AS start," +
                                    " type(r) AS relationship1," +
                                    " r.pmid AS pmid1," +
                                    " length(r.pmid) AS npmid1," +
                                    " endnode(r).name AS middle," +
                                    " type(r2) AS relationship2," +
                                    " r2.pmid AS pmid2," +
                                    " length(r2.pmid) AS npmid2," +
                                    " endnode(r2).name AS end").data())

In [64]:
# Get the indirect incoming paths
incoming = pd.DataFrame(session.run("Match p = (x:Protein)-[r]->(y:Protein)-[r2]->(n)"
                                    " WHERE n.name IN " + str(list(set(farashi["Target/assigned/e-Gene"]))) +
                                    " RETURN startnode(r).name AS start," +
                                    " type(r) AS relationship1," +
                                    " r.pmid AS pmid1," +
                                    " length(r.pmid) AS npmid1," +
                                    " endnode(r).name AS middle," +
                                    " type(r2) AS relationship2," +
                                    " r2.pmid AS pmid2," +
                                    " length(r2.pmid) AS npmid2," +
                                    " endnode(r2).name AS end").data())

In [65]:
# Create features based on paths

In [ ]:
## Get shapes
# Triangles query
# Not exists clauses toevoegen
# nodes laten returnen om unieke combinaties te filteren
triangles = "Match p = (n)--(i:Protein)--(i2:Protein)--(n) WHERE n.name = '" + 'KLRK1' + "' RETURN nodes(p)"
#squares = "Match p = (n)--(i:Protein)--(i2:Protein)--(i3:Protein)--(n) WHERE n.name = '" +  + "' RETURN COUNT(p)"
#diagonalSquares = "Match p = (n)-[r]-(i:Protein)-[r2]-(i2:Protein)-[r3]-(i3:Protein)-[r4]-(n)-[r5]-(i2:Protein) WHERE n.name = '" +  + "' RETURN COUNT(p)"
#diagonalSquares2 = "Match p = (i3:Protein)-[r5]-(n)-[r]-(i:Protein)-[r2]-(i2:Protein)-[r3]-(i3:Protein)-[r4]-(i:Protein) WHERE n.name = '" +  + "' RETURN COUNT(p)"

test = session.run(triangles)

In [ ]:
# Calculate network statistics such as eigenvector centrality 
# TODO: Alleen genen nemen die tot expressie komen, en alle relaties includeren
eigenvector = "CALL gds.alpha.eigenvector.stream({" +
              " nodeProjection: 'Protein'," +
              " relationshipProjection: 'Regulation'" +
              "})" + 
              " YIELD nodeId, score" + 
              " RETURN gds.util.asNode(nodeId).name AS page, score" + 
              " ORDER BY score DESC"

degree = "CALL gds.alpha.degree.stream({" +
              " nodeProjection: 'Protein'," +
              " relationshipProjection: 'Regulation'" +
              "})" + 
              " YIELD nodeId, score" + 
              " RETURN gds.util.asNode(nodeId).name AS page, score" + 
              " ORDER BY score DESC"

## Query pathway studio for information about Prostate cancer and it's associated pathways, and use it to filter the pathway information

In [11]:
bla = session.run("MATCH (n:Disease) RETURN n.name ORDER BY n.name DESC")

In [15]:
import re

r = re.compile("^prostate.*")
list(filter(r.match, out))

['prostate neoplasm', 'prostate carcinoma', 'prostate cancer']

In [23]:
# Extract the disease nodes for prostate cancer
pca_names = ['prostate neoplasm',
             'prostate carcinoma',
             'prostate cancer']

pca = session.run("MATCH (n:Disease) WHERE n.name IN " + str(pca_names) + " RETURN n.name, id(n)").data()
pca = dict(zip([x["n.name"] for x in pca], [x["id(n)"] for x in pca]))

In [43]:
# Find relationships between the disease nodes --> pathways --> genes
test = session.run("MATCH (n)-[r]->(path:Pathway)<-[r2]-(prot:Protein) WHERE id(n) IN " + str(list(pca.values())) + " RETURN path.name AS pathway, prot.name AS gene").data()

test2 = session.run("MATCH (n)-[r]->(prot:Protein) WHERE id(n) IN " + str(list(pca.values())) + " RETURN type(r) AS relationship, r.pmid AS pmid, prot.name AS gene").data()